# 0. Grid setup

In this notebook, we create a hex grid from the map of Montreal.
The grid cell is the unit of observation, and each cell is assigned spatial features based on Points of Interest and demographic measures.
The goal of this exploration is to see if clustering of these features can align with geographic clustering in a meaningful way.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import geohexgrid as ghg
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from final_project.config import CRS, DATA_DIR, HEX_RADIUS, RAW_DIR
from final_project.utils import create_map_plot, save_figure

sns.set_theme(style='white', palette='Set1')

The island shapefile and administrative boundaries shapefile should be extracted to the data directory with the required names.

In [ ]:
island_shp = RAW_DIR / 'island' / 'limites-terrestres.shp'
agglo_shp = RAW_DIR / 'city' / 'limites-administratives-agglomeration.shp'

The grid itself is generated by [Geohexgrid](https://github.com/araichev/geohexgrid/tree/master) using the geographic boundaries of the Island of Montreal and minor outlying islands.
The hex circumradius is set to 250 m for a suitable granularity; there are 3,371 cells, with each one representing approximately 0.15 sq km.

In [ ]:
# Load the island shapefile.
island = gpd.read_file(island_shp)
island = island.to_crs(CRS)

# Make the grid overlay from the island boundaries.
grid = (ghg.make_grid_from_gdf(island, R=HEX_RADIUS)
           .assign(cell_id=lambda gdf: range(len(gdf)))
           .set_index('cell_id'))
print(f"Number of grid cells: {grid.shape[0]}")

We are analyzing the Island of Montreal, which contains the city of Montreal and several other independent municipalities.
However, when people refer to "Montreal," they are usually referring to the continuous urban agglomeration that is the entire island.
Therefore, we load the administrative boundaries of the independent municipalities and of Montreal's constitutent boroughs, which resemble cities with their own mayors and councils.

In [ ]:
# Load the subdivisions.
agglo = (gpd.read_file(agglo_shp)
            .loc[:,['geometry', 'NOM']]
            .rename(columns={'NOM': 'subdivision'})
            .to_crs(CRS))

# Assign each cell the borough or municipality that its centroid lies in.
grid_pts = (grid.copy()
                .assign(pt=lambda gdf: gdf.geometry.centroid)
                .set_geometry('pt'))
grid_pts = gpd.sjoin(grid_pts, agglo, how='left', predicate='within')
grid = (grid.assign(subdivision=grid_pts['subdivision'].values)
            .dropna())
print(f"Number of grid cells: {grid.shape[0]}")

In [ ]:
fig, ax = create_map_plot('Grid representation of Montreal')
grid.plot(ax=ax, facecolor='skyblue', edgecolor='dimgrey', alpha=0.67)
grid.dissolve(by='subdivision').boundary.plot(ax=ax, color='black')
plt.tight_layout()
plt.show()

save_figure(fig, 'montreal_hex_grid')

Save this grid for feature engineering in the next steps.

In [ ]:
grid.to_file(DATA_DIR / 'grid.geojson', driver='GeoJSON')